In [1]:
import spacy
from spacy import displacy
import pickle
activated = spacy.prefer_gpu()
nlp = spacy.load('en_core_web_lg')

In [26]:
test_case = [
    "Understand customer needs and develop high-quality presentations, proposals and software demonstrations that speak to these needs - building a consensus for change on a multi-stakeholder basis",
    "Work with the Enterprise sales team to build, manage and maintain customer relationships",
    "Need to understand and demonstrate the solution and how to integrate to other business systems",
    "Understanding and experience within KPIs, identifying data sources, building a visualization map and dashboarding.",
    "Define, guide and implement innovation programs' governance, portfolio mix and business fit",
    "Establish the use of cloud services as a means to drive innovation",
    "Collaborate with other AWS specialist teams to drive and shape solutions to accelerate our customer's business outcomes",
    "Experience with a major consulting firm or experience in a corporate role delivering strategy and operations projects with an emphasis in business transformation at the intersection of new operating models and new digital technologies.",
    "Understanding of modern cloud delivery models (Public, Private and Hybrid).",
    "Excellent program management skills including developing project plans, resourcing and budgeting projects, and managing a disciplined execution methodology with both internal (direct) and external (indirect) team members.",
    "Experience in two or more of the following transformation practice areas: ERP (business process architecture and solution implementation), business case and use case development, financial modelling, BI and Analytics strategy and implementation, complex program management, enterprise business / process / IT architectures, executive agenda items and metrics."
]

In [138]:
# preprocess such as remove parenthesis
def preprocess(text):
    return text.replace("(","").replace(")", "")

# Split VP phrases from the sentence
def split_VP_phrases(doc):
    print("Base ===== ", doc.text, " =======\n")
    res = []
    root = doc[:].root
    
    visited = [False for _ in doc ]
    conj_list = [None for _ in doc]
    
    def convert(x):
        return x
    
    # check if rt is conjunction
    def check_conj(rt):
        if conj_list[rt.i] != None:
            return conj_list[rt.i]
        if rt.i < rt.head.i:
            conj_list[rt.i] = (False, None)
            return False, None
        for token in rt.rights:
            if rt.pos_ != 'VERB' and token.dep_ not in ['cc', 'conj', 'punct']:
                conj_list[rt.i] = (False, None)
                return False, None
            if token.dep_ == 'conj':
                conj_list[rt.i] = (True, token)
                return True, token
        for token in rt.head.rights:
            if token.dep_ in ['ccomp', 'xcomp'] and rt.i < token.i:
                token.dep_ = 'conj'
                conj_list[rt.i] = (True, token)
                return True, token
        conj_list[rt.i] = (False, None)
        return False, None
    
    # check if rt is the root of VP(verb phrase)
    def check_split_verb(rt):
        if rt.pos_ != 'VERB':
            return False
        if rt.dep_ == 'conj':
            if rt.head.pos_ != 'VERB':
                return True
            return check_split_verb(rt.head)
        if rt.dep_ in ['xcomp', 'ccomp', 'advcl', 'nsubj', 'acl', 'relcl', 'prep', 'pcomp']:
            return False
        return True
    
    # check if rt is the punction
    def check_punct(rt):
        if rt.pos_ == 'PUNCT' or rt.pos_ == 'SYM':
            return True
        if rt.pos_ == 'CCONJ':
            is_conj, _ = check_conj(rt.head)
            return is_conj
        return False
    
    # split to phrases
    def split(rt):  
        visited[rt.i] = True
    
        if check_punct(rt):
            return []
        
        ret = ['']
        if not check_split_verb(rt) or rt == root:
            for token in rt.lefts:
                left_ret = split(token)
                if len(left_ret) == 0:
                    continue
                ret_ = [x + y + " " for x in ret for y in left_ret]
                ret = ret_
     
        ret_l = ret
        ret_ = [x + rt.text for x in ret]
        ret = []
     
        flag = False
        extra_flag = False
        is_conj, conj = check_conj(rt)
        
        rights = [tk for tk in rt.rights]
        if is_conj and not conj in rights:
            rights.append(conj)
        
        for token in rights:
            if (visited[token.i]):
                continue
            right_ret = split(token)
            if len(right_ret) == 0:
                continue
            
            if is_conj and token.dep_ == 'conj':
                if not flag and token.pos_ == rt.pos_:
                    if not extra_flag:
                        for sent in right_ret:
                            right = doc[token.left_edge.i : token.i+1].text
                            if right in sent:
                                ret.extend([sent.replace(right, x) for x in ret_])
                    flag = True
                if not check_split_verb(token):
                    if rt.pos_ == 'VERB':
                        ret.extend([x + y for x in ret_l for y in right_ret])
                    else:
                        ret.extend(right_ret)
                if not flag and token == conj:
                    ret.extend(ret_)
                ret_ = ret
               
            else:
                extra_flag = True
                ret = [x + " " + y for x in ret_ for y in right_ret]
                ret_ = ret
       
        ret = ret_
        
        if check_split_verb(rt):
            res.extend(ret)
            is_conj, conj = check_conj(rt.head)
            if is_conj and conj == rt:
                return ret
            else:
                return []
        else:
            if rt.i == root.i:
                res.extend(ret)
            return ret
        
    split(root)
    
    # Normalize the sentence
    res_ = []
    for sent in res:
        sent = nlp(sent)
        cnt = 0
        for token in sent:
            if token.pos_ in ['VERB', 'NOUN']:
                cnt += 1
        if cnt < 2:
            continue    
        res_.append(sent.text)
    res = convert(res_)
    print('\n'.join(res))
    return res

ex = nlp(preprocess(test_case[10]))

from spacy import displacy
displacy.render(ex, style = "dep", jupyter = True)
ret = split_VP_phrases(ex)

Base =====  Experience in two or more of the following transformation practice areas: ERP business process architecture and solution implementation, business case and use case development, financial modelling, BI and Analytics strategy and implementation, complex program management, enterprise business / process / IT architectures, executive agenda items and metrics.  =======

use case development
use financial modelling
use BI and Analytics strategy
use implementation
use enterprise business process IT architectures
use executive agenda items
use metrics
Experience in more of the following transformation practice areas ERP business process architecture business case and
Experience in more of the following transformation practice areas solution implementation business case and
Experience in two of the following transformation practice areas ERP business process architecture business case and
Experience in two of the following transformation practice areas solution implementation busine

In [12]:
for token in ex:
    print(token.dep_)

ROOT
cc
conj
prep
pobj
punct
acl
compound
dobj
punct
conj
det
compound
dobj
cc
conj


In [14]:
spacy.explain("ADP")

'adposition'

In [2]:
import textacy

In [3]:
example = "Textacy is a Python library for performing higher-level natural language processing (NLP) tasks, built on the high-performance Spacy library. With the basics — tokenization, part-of-speech tagging, parsing — offloaded to another library, textacy focuses on tasks facilitated by the availability of tokenized, POS-tagged, and parsed text: keyterm extraction, readability statistics, emotional valence analysis, quotation attribution, and more."

In [12]:
raw_text = """Experience in two or more of the following transformation practice areas: ERP (business process architecture and solution implementation), business case and use case development, financial modelling, BI and Analytics strategy and implementation, complex program management, enterprise business / process / IT architectures, executive agenda items and metrics."""

In [13]:
raw_text

'Experience in two or more of the following transformation practice areas: ERP (business process architecture and solution implementation), business case and use case development, financial modelling, BI and Analytics strategy and implementation, complex program management, enterprise business / process / IT architectures, executive agenda items and metrics.'

In [16]:
doc = textacy.make_spacy_doc(raw_text)

In [15]:
from textacy import preprocessing

In [17]:
doc._.preview

'Doc(57 tokens: "Experience in two or more of the following tran...")'

In [19]:
pattern = textacy.constants.POS_REGEX_PATTERNS["en"]["NP"]
pattern

'<DET>? <NUM>* (<ADJ> <PUNCT>? <CONJ>?)* (<NOUN>|<PROPN> <PART>?)+'

In [22]:
ts = textacy.TextStats(doc)

In [23]:
ts.readability_stats

{'flesch_kincaid_grade_level': 27.395555555555564,
 'flesch_reading_ease': -21.19999999999996,
 'smog_index': 25.98008509911553,
 'gunning_fog_index': 32.22222222222222,
 'coleman_liau_index': 22.871370911111114,
 'automated_readability_index': 32.574666666666666,
 'lix': 100.55555555555556,
 'gulpease_index': 28.77777777777778,
 'wiener_sachtextformel': 19.281222222222222}

In [24]:
ts.basic_counts

{'n_sents': 1,
 'n_words': 45,
 'n_chars': 301,
 'n_syllables': 97,
 'n_unique_words': 36,
 'n_long_words': 25,
 'n_monosyllable_words': 20,
 'n_polysyllable_words': 16}

In [4]:
import textacy.resources
rs = textacy.resources.ConceptNet()
rs.download()
rs.info

{'name': 'concept_net',
 'site_url': 'http://conceptnet.io',
 'publication_url': 'https://arxiv.org/abs/1612.03975',
 'description': 'An open, multilingual semantic network of general knowledge, designed to help computers understand the meanings of words.'}

In [10]:
rs.get_antonyms("go", lang="en", sense="v")

757it [00:00, 84333.93it/s]


ValueError: not enough values to unpack (expected 5, got 4)